In [1]:
#!pip install numpy pandas
#!pip install scikit-learn
#!pip install tensorflow
#!pip install seaborn matplotlib

In [2]:
import re
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob


In [3]:
# BLOQUE 1: FUNCIONES PARA PARSEAR EL ARCHIVO INPUT (Features)

def _normalizar_clave(clave):
    """Normaliza una clave reemplazando caracteres especiales."""
    return clave.replace('(', '').replace(')', '').replace(' ', '')

def _procesar_valor_numerico(valor, clave, aplicar_limites, limites_por_columna, umbral_cero):
    """
    Procesa un valor numérico, manejando casos especiales como inf/nan
    y aplicando límites si es necesario.
    """
    # Manejar valores especiales
    valor_lower = valor.lower()
    if 'inf' in valor_lower:
        num_valor = np.inf
    elif 'nan' in valor_lower:
        num_valor = np.nan
    else:
        num_valor = float(valor)

    # Convertir valores muy pequeños a cero
    if np.isfinite(num_valor) and abs(num_valor) < umbral_cero:
        num_valor = 0.0

    # Aplicar límites si está habilitado
    if aplicar_limites and clave in limites_por_columna:
        lim_inf, lim_sup = limites_por_columna[clave]
        if lim_sup is not None and num_valor > lim_sup:
            print(f"Valor 'inf' o muy alto en '{clave}' recortado a {lim_sup}")
            num_valor = lim_sup
        if lim_inf is not None and num_valor < lim_inf:
            num_valor = lim_inf

    return num_valor

def parse_input_data(ruta_archivo, aplicar_limites=False):
    LIMITES_POR_COLUMNA = {
        "cota_inferior": (-1e7, 1e7),     # Equivalente a 'lb_f_obj'
        "cota_superior": (None, 10000000.0), # Equivalente a 'ub_f_obj'
        "diametro_grande": (None, 10000000.0)  # Equivalente a 'bigger_diam'
    }
    UMBRAL_CERO = 1e-7

    with open(ruta_archivo, 'r') as file:
        lineas = file.readlines()

    todos_los_registros = []
    registro_actual = {}
    
    benchmark_actual = "default_benchmark" 

    for linea in lineas:
        linea_limpia = linea.strip()

        if linea_limpia.endswith('.bch'):
            benchmark_actual = linea_limpia
            continue

        # Procesar final de registro (línea en blanco)
        if not linea_limpia:
            if registro_actual and benchmark_actual:
                registro_actual['benchmark'] = benchmark_actual
                todos_los_registros.append(registro_actual)
                registro_actual = {}
            continue

        # Procesar pares "clave: valor"
        match = re.match(r'([^:]+?)\s*:\s*(.*)', linea_limpia)
        if not match:
            continue

        clave, valor = match.groups()
        clave = _normalizar_clave(clave.strip()) # Normaliza 'cota_superior'
        valor = valor.strip()
        
        # Intentar conversión a entero primero
        try:
            registro_actual[clave] = int(valor)
            continue
        except ValueError:
            pass

        # Intentar conversión a float y aplicar procesamiento
        try:
            num_valor = _procesar_valor_numerico(valor, clave, aplicar_limites,
                                                 LIMITES_POR_COLUMNA, UMBRAL_CERO)
            registro_actual[clave] = num_valor
        except ValueError:
            # Mantener como string si no es numérico
            registro_actual[clave] = valor

    # Añadir último registro si el archivo no termina con línea en blanco
    if registro_actual and benchmark_actual:
        registro_actual['benchmark'] = benchmark_actual
        todos_los_registros.append(registro_actual)

    return todos_los_registros


In [4]:
# BLOQUE 2: FUNCIÓN PARA PARSEAR EL ARCHIVO OUTPUT (Heurísticas)
def parse_output_data(ruta_archivo):
    """
    Procesa un archivo de benchmark que contiene los resultados de las
    técnicas (ej. LSMEAR, LF, etc.).
    Estos archivos se caracterizan por tener pares clave-valor separados por espacios.
    """
    with open(ruta_archivo, 'r') as file:
        lineas = file.readlines()

    todos_los_registros = []
    registro_actual = {}
    # Misma asunción sobre benchmark
    benchmark_actual = "default_benchmark" 

    for linea in lineas:
        linea_limpia = linea.strip()

        # Identificar línea de benchmark
        if linea_limpia.endswith('.bch'):
            benchmark_actual = linea_limpia
            continue
            
        # Procesar final de registro (línea en blanco)
        if not linea_limpia:
            if registro_actual and benchmark_actual:
                registro_actual['benchmark'] = benchmark_actual
                todos_los_registros.append(registro_actual)
                registro_actual = {}
            continue

        # Procesar pares "CLAVE VALOR"
        parts = linea_limpia.split(maxsplit=1)
        if len(parts) == 2:
            clave, valor = parts
            clave = clave.strip()

            # Normalizar clave 'id:'
            if clave == 'id:':
                clave = 'id'

            try:
                registro_actual[clave] = int(valor.strip())
            except ValueError:
                registro_actual[clave] = valor.strip()

    # Añadir último registro
    if registro_actual and benchmark_actual:
        registro_actual['benchmark'] = benchmark_actual
        todos_los_registros.append(registro_actual)

    return todos_los_registros


# Procesamiento de Múltiples Archivos y Enriquecimiento del Dataset

Este bloque permite procesar múltiples parejas de archivos input/output y agregar todos los datos a un **dataframe único y acumulativo**.

In [5]:
# BLOQUE 3: PROCESAMIENTO MÚLTIPLE Y ENRIQUECIMIENTO DE DATASET

def emparejar_archivos(input_dir, output_dir):
    """
    Empareja automáticamente archivos input/output basándose en nombres similares.
    Maneja variaciones de nombres comunes.
    """
    archivos_input = sorted(glob.glob(input_dir + '*.txt'))
    archivos_a_procesar = []
    
    for input_file in archivos_input:
        basename = os.path.basename(input_file)
        basename_sin_ext = basename.replace('.txt', '')
        
        # Busca el output con el mismo nombre
        output_file = os.path.join(output_dir, basename)
        
        # Si no existe, intenta variaciones
        if not os.path.exists(output_file):
            # Reemplaza "input" por "output" en el nombre
            output_basename = basename_sin_ext.replace('input', 'output')
            output_file = os.path.join(output_dir, output_basename + '.txt')
        
        if not os.path.exists(output_file):
            # Intenta removiendo sufijos como "-1", "-2"
            output_basename = os.path.splitext(basename_sin_ext)[0].rsplit('-', 1)[0]
            output_file = os.path.join(output_dir, output_basename + '.txt')
        
        if os.path.exists(output_file):
            archivos_a_procesar.append({
                'input': input_file,
                'output': output_file
            })
            print(f"✓ Emparejado: {os.path.basename(input_file)} → {os.path.basename(output_file)}")
        else:
            print(f"✗ No encontrado output para: {basename}")
    
    return archivos_a_procesar

# 1. Define rutas
input_dir = '/home/cristopher/Escritorio/libreria/dataset/input/'
output_dir = '/home/cristopher/Escritorio/libreria/dataset/output/'

# 2. Empareja automáticamente
archivos_a_procesar = emparejar_archivos(input_dir, output_dir)

print(f"\nTotal de parejas: {len(archivos_a_procesar)}\n")




✓ Emparejado: easy_ex2_1_1.txt → easy_ex2_1_1.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_ex2_1_7.txt → optimizer_output_benchs_victor_fixxed_ex2_1_7.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_ex2_1_8.txt → optimizer_output_benchs_victor_fixxed_ex2_1_8.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_himmelbk.txt → optimizer_output_benchs_victor_fixxed_himmelbk.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_hs056.txt → optimizer_output_benchs_victor_fixxed_hs056.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_hs070.txt → optimizer_output_benchs_victor_fixxed_hs070.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_hs087.txt → optimizer_output_benchs_victor_fixxed_hs087.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_hs088.txt → optimizer_output_benchs_victor_fixxed_hs088.txt
✓ Emparejado: optimizer_input_benchs_victor_fixxed_hs089.txt → optimizer_output_benchs_victor_fixxed_hs089.txt
✓ Emparejado: optimizer_input_benchs_victor_fixx

In [6]:
# 2. Inicializa el dataframe final (vacío al inicio)
df_final = pd.DataFrame()

print("="*60)
print("INICIANDO PROCESAMIENTO DE MÚLTIPLES ARCHIVOS")
print("="*60)

# 3. Procesa cada pareja de archivos
for idx, archivo_pareja in enumerate(archivos_a_procesar, 1):
    ruta_input = archivo_pareja['input']
    ruta_output = archivo_pareja['output']
    
    # Verifica que los archivos existan
    if not os.path.exists(ruta_input):
        print(f"\n [{idx}] ADVERTENCIA: Archivo input NO ENCONTRADO: {ruta_input}")
        continue
    if not os.path.exists(ruta_output):
        print(f"\n [{idx}] ADVERTENCIA: Archivo output NO ENCONTRADO: {ruta_output}")
        continue
    
    print(f"\n--- Procesando pareja [{idx}] ---")
    print(f"Input:  {os.path.basename(ruta_input)}")
    print(f"Output: {os.path.basename(ruta_output)}")
    
    try:
        # 4. Parsea los archivos
        datos_input = parse_input_data(ruta_input, aplicar_limites=True)
        datos_output = parse_output_data(ruta_output)
        
        # 5. Convierte a DataFrames
        df_input = pd.DataFrame(datos_input)
        df_output = pd.DataFrame(datos_output)
        
        print(f"  Filas input: {len(df_input)}")
        print(f"  Filas output: {len(df_output)}")
        
        # 6. Une los DataFrames
        df_merged_local = pd.merge(df_input, df_output, on=['benchmark', 'id'])
        print(f"  Filas después del merge: {len(df_merged_local)}")
        
        # 7. Limpieza de datos
        columnas_tecnicas = ['LSMEAR', 'LF', 'RR', 'SM', 'SS', 'SSR']
        
        for columna in columnas_tecnicas:
            if columna not in df_merged_local.columns:
                df_merged_local[columna] = -1
            else:
                df_merged_local[columna] = df_merged_local[columna].fillna(-1)
        
        df_merged_local[columnas_tecnicas] = df_merged_local[columnas_tecnicas].astype(int)
        
        # 8. Ordena las columnas
        features_reales = ['variables', 'restricciones', 'profundidad', 'cota_superior', 
                          'cota_inferior', 'diametro_grande', 'diametro_pequeno']
        
        ordered_columns = ['benchmark', 'id'] + features_reales + columnas_tecnicas
        df_merged_local = df_merged_local.reindex(columns=ordered_columns)
        
        # 9. ACUMULA EN df_final (concatena)
        df_final = pd.concat([df_final, df_merged_local], ignore_index=False)
        print(f"  ✓ Agregado a dataset. Total acumulado: {len(df_final)}")
        
    except Exception as e:
        print(f"  ✗ ERROR procesando esta pareja: {str(e)}")
        continue

print("\n" + "="*60)
print(f"PROCESAMIENTO COMPLETADO")
print("="*60)
print(f"Filas totales en df_final: {len(df_final)}")
print(f"\nPrimeras filas del dataset enriquecido:")
print(df_final.head())

print(f"\nVerificación de NaN:")
print(df_final.isna().sum())


INICIANDO PROCESAMIENTO DE MÚLTIPLES ARCHIVOS

--- Procesando pareja [1] ---
Input:  easy_ex2_1_1.txt
Output: easy_ex2_1_1.txt
  Filas input: 38
  Filas output: 38
  Filas después del merge: 38
  ✓ Agregado a dataset. Total acumulado: 38

--- Procesando pareja [2] ---
Input:  optimizer_input_benchs_victor_fixxed_ex2_1_7.txt
Output: optimizer_output_benchs_victor_fixxed_ex2_1_7.txt
  Filas input: 280
  Filas output: 280
  Filas después del merge: 280
  ✓ Agregado a dataset. Total acumulado: 318

--- Procesando pareja [3] ---
Input:  optimizer_input_benchs_victor_fixxed_ex2_1_8.txt
Output: optimizer_output_benchs_victor_fixxed_ex2_1_8.txt
  Filas input: 227
  Filas output: 227
  Filas después del merge: 227
  ✓ Agregado a dataset. Total acumulado: 545

--- Procesando pareja [4] ---
Input:  optimizer_input_benchs_victor_fixxed_himmelbk.txt
Output: optimizer_output_benchs_victor_fixxed_himmelbk.txt
Valor 'inf' o muy alto en 'cota_superior' recortado a 10000000.0
Valor 'inf' o muy alto en '

In [7]:
# BLOQUE 3.5: GUARDAR Y CARGAR DATASET EN EXCEL

nombre_archivo_excel = 'dataset.xlsx'

# Guardar el dataset actual
df_final.to_excel(nombre_archivo_excel, index=False)
print(f"✓ Dataset guardado en: {nombre_archivo_excel}")
print(f"  Total de filas: {len(df_final)}")


✓ Dataset guardado en: dataset.xlsx
  Total de filas: 33965


### Calculo de probabilidades

In [ ]:
# 1. DEFINIR FEATURES Y TÉCNICAS

features = ['variables', 'restricciones', 'profundidad', 'cota_superior', 
            'cota_inferior', 'diametro_grande', 'diametro_pequeno']
techniques = ['LSMEAR', 'LF', 'RR', 'SM', 'SS', 'SSR']

print(f"Dataset original (df_final): {len(df_final)} filas")

# 2. FILTRA FILAS INVÁLIDAS (redundante?)
filas_validas = (df_final[techniques] != -1).all(axis=1)
df_final_limpio = df_final[filas_validas].copy()

print(f"Dataset filtrado (sin -1): {len(df_final_limpio)} filas")

# 3. SEPARA X (FEATURES) E Y (TARGETS BRUTOS)
X = df_final_limpio[features]
Y_raw = df_final_limpio[techniques]


In [ ]:
# 4. FUNCIÓN PARA CALCULAR PROBABILIDADES 
def calcular_probabilidades_por_promedio(nodos):
    # 1. Filtrar valores válidos
    nodos_validos = nodos[nodos >= 0]
    if nodos_validos.empty:
        return pd.Series(0.0, index=nodos.index)

    # 2. Calcular el PROMEDIO
    promedio = nodos_validos.mean()

    # 3. Filtrar: Solo sobreviven las técnicas MEJORES (menores) al promedio
    calificadas_mask = (nodos_validos <= promedio)
    tecnicas_calificadas = nodos_validos[calificadas_mask]
    
    probabilidades = pd.Series(0.0, index=nodos.index)

    # Caso especial: Si ninguna califica (raro) o empate total
    if tecnicas_calificadas.empty:
        valor_minimo = nodos_validos.min()
        mejores_tecnicas_mask = (nodos_validos == valor_minimo)
        num_ganadores = mejores_tecnicas_mask.sum()
        if num_ganadores > 0:
            probabilidades[mejores_tecnicas_mask] = 1.0 / num_ganadores
        return probabilidades

    # 4. Calcular Mérito: Distancia desde el promedio
    # (Promedio - Valor) -> Mientras más chico el valor, más grande el mérito
    merito = promedio - tecnicas_calificadas

    suma_meritos = merito.sum()

    # 5. Normalizar (Crear porcentajes)
    if suma_meritos > 0:
        probabilidades.update(merito / suma_meritos)
    else:
        # Si todas las calificadas son iguales al promedio
        num_calificadas = len(tecnicas_calificadas)
        probabilidades[tecnicas_calificadas.index] = 1.0 / num_calificadas

    return probabilidades


In [ ]:

# 5. APLICA PARA CREAR VERDADERA 'y'
# .apply(..., axis=1) aplica la función a CADA FILA del DataFrame
y = Y_raw.apply(calcular_probabilidades_por_promedio, axis=1)
print("Variable objetivo 'y' (probabilidades) creada")

# 6. VERIFICA EL RESULTADO
print("\n--- Ejemplo de Y_raw (Resultados Brutos) ---")
print(Y_raw.head())

print("\n--- Ejemplo de 'y' (Probabilidades Objetivo - Método 'Peor Valor') ---")
print(y.head())

print("\n--- Verificación: la suma de probabilidades por fila debe ser 1 (o 0) ---")
print(y.sum(axis=1).head())